In [43]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare


In [16]:
#importing data
#data exploration

jeopardy=pd.read_csv('jeopardy.csv')

print(jeopardy.head(5))
jeopardy.columns

   Show Number    Air Date      Round                         Category  Value  \
0         4680  2004-12-31  Jeopardy!                          HISTORY   $200   
1         4680  2004-12-31  Jeopardy!  ESPN's TOP 10 ALL-TIME ATHLETES   $200   
2         4680  2004-12-31  Jeopardy!      EVERYBODY TALKS ABOUT IT...   $200   
3         4680  2004-12-31  Jeopardy!                 THE COMPANY LINE   $200   
4         4680  2004-12-31  Jeopardy!              EPITAPHS & TRIBUTES   $200   

                                            Question      Answer  
0  For the last 8 years of his life, Galileo was ...  Copernicus  
1  No. 2: 1912 Olympian; football star at Carlisl...  Jim Thorpe  
2  The city of Yuma in this state has a record av...     Arizona  
3  In 1963, live on "The Art Linkletter Show", th...  McDonald's  
4  Signer of the Dec. of Indep., framer of the Co...  John Adams  


Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')

In [17]:
jeopardy.columns = ['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question', 'Answer']

In [18]:
jeopardy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19999 entries, 0 to 19998
Data columns (total 7 columns):
Show Number    19999 non-null int64
Air Date       19999 non-null object
Round          19999 non-null object
Category       19999 non-null object
Value          19999 non-null object
Question       19999 non-null object
Answer         19999 non-null object
dtypes: int64(1), object(6)
memory usage: 1.1+ MB


In [19]:
#data cleaning
import re

def normalize_text(text):
    text = re.sub("[^A-Za-z0-9\s]", "", text.lower())
    text = re.sub("\s+", " ", text)
    return text

def normalize_values(text):
    text = re.sub("[^A-Za-z0-9\s]", "", text)
    print(text)
    try:
        text = int(text)
    except Exception:
        text = 0
    return text

In [20]:
jeopardy['clean_value']=jeopardy['Value'].apply(normalize_values)
jeopardy['clean_question']=jeopardy['Question'].apply(normalize_text)
jeopardy['clean_answer']=jeopardy['Answer'].apply(normalize_text)
jeopardy['Air Date']=pd.to_datetime(jeopardy['Air Date'])

200
200
200
200
200
200
400
400
400
400
400
400
600
600
600
600
600
600
800
800
800
800
2000
800
1000
1000
1000
1000
1000
400
400
400
400
400
400
800
800
800
800
800
1200
2000
1200
1200
1200
1600
1600
1600
1600
1600
2000
2000
3200
2000
2000
None
200
200
200
200
200
200
400
400
400
400
400
400
600
600
600
600
600
600
800
800
800
800
800
800
2000
1000
1000
1000
1000
1000
400
400
400
400
400
400
800
800
800
800
800
800
1200
1200
1200
1200
1200
1200
5000
1600
1600
1600
1600
5000
2000
2000
2000
2000
2000
2000
None
100
100
100
100
100
100
200
200
200
200
200
200
300
300
300
300
300
300
400
400
500
400
400
400
500
500
500
500
500
200
200
200
200
200
200
400
400
400
400
400
400
600
600
600
600
600
600
800
1000
800
2000
800
1000
1000
1000
1000
1000
None
100
100
100
100
100
100
200
200
200
200
200
200
300
300
300
300
300
300
400
400
400
400
400
400
500
500
500
500
800
500
200
200
200
200
200
200
400
400
400
400
400
400
600
600
600
600
600
600
800
800
1500
800
800
800
1000
1000
1000
1000
1000
100

In [21]:
def string_clean(row):
    split_answer=row['clean_answer'].split()
    split_question=row['clean_question'].split()
    match_count=0
    if 'the' in split_answer:
        split_answer.remove('the')
    if len(split_answer)==0:
        return(0)
    for i in split_answer:
        if i in split_question:
            match_count+=1
    return(match_count/len(split_answer))

#counting number of times terms in clean_answer occur in clean_question

jeopardy['answer_in_question']=jeopardy.apply(string_clean,axis=1)

mean_matches_ratio=jeopardy['answer_in_question'].mean()
print(mean_matches_ratio)



    
    

0.05900196524977763


On average, the answer words makes up for about 6% of the question. This is not significant, and it will be difficult to figure out answer by the question itself. We'll probably have to study deeper.

In [22]:
#checking if terms in questions have been used previously

question_overlap=[]
terms_used=set()

jeopardy=jeopardy.sort_values('Air Date')

for i,row in jeopardy.iterrows():
    split_question=row['clean_question'].split(" ")
    split_question=[j for j in split_question if len(j)>5]
    match_count=0
    for k in split_question:
        if k in terms_used:
            match_count+=1
    for l in split_question:
        terms_used.add(l)
    if len(split_question)>0:
        match_count/=len(split_question)
    question_overlap.append(match_count)
    
jeopardy['question_overlap']=question_overlap
print(jeopardy['question_overlap'].mean())
    
    
    

0.6876260592169802


## Low v/s High value questions
There is almost 70% overlap between terms of new and older questions. Therefore recycling of questions can be explored more.

In [23]:
def value_count(row):
    value=0
    if row['clean_value']>800:
        value=1
    else:
        value=0
    return value

#determining question with high and low value
jeopardy['high_value']=jeopardy.apply(value_count,axis=1)



In [28]:
def word_count(word):
    low_count=0
    high_count=0
    for i,row in jeopardy.iterrows():
        split_term=row['clean_question'].split(" ")
        if word in split_term:
            if row['high_value']==1:
                high_count+=1
            else:
                low_count+=1
    return high_count,low_count   
        

In [29]:
from random import choice

terms_used_list=list(terms_used)
comparison_terms=[choice(terms_used_list) for _ in range(10)]
observed_expected=[]

for term in comparison_terms:
    observed_expected.append(word_count(term))
    
observed_expected
    

[(0, 1),
 (0, 1),
 (0, 1),
 (2, 6),
 (1, 0),
 (1, 2),
 (1, 0),
 (0, 3),
 (0, 3),
 (1, 0)]

In [36]:
#after finding observed counts for the ten terms, I am finding expected counts and chi-squared values

high_value_count=len(jeopardy[jeopardy['high_value']==1])
low_value_count=len(jeopardy[jeopardy['high_value']==0])

In [35]:
high_value_count

5734

In [37]:
low_value_count

14265

In [44]:
chi_squared=[]

for lst in observed_expected:
    total=lst[0]+lst[1]
    total_prop=total/len(jeopardy)
    high_value_exp = total_prop * high_value_count
    low_value_exp = total_prop * low_value_count
    
    observed = np.array([lst[0], lst[1]])
    expected = np.array([high_value_exp, low_value_exp])
    chi_squared.append(chisquare(observed, expected))

    

In [45]:
chi_squared

[Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=0.05272886616881538, pvalue=0.818381104912348),
 Power_divergenceResult(statistic=2.487792117195675, pvalue=0.11473257634454047),
 Power_divergenceResult(statistic=0.03188116723440362, pvalue=0.8582887163235293),
 Power_divergenceResult(statistic=2.487792117195675, pvalue=0.11473257634454047),
 Power_divergenceResult(statistic=1.205888538380652, pvalue=0.27214791766902047),
 Power_divergenceResult(statistic=1.205888538380652, pvalue=0.27214791766902047),
 Power_divergenceResult(statistic=2.487792117195675, pvalue=0.11473257634454047)]

## Chi-Squared Results
Since p value is high, the terms have no significant difference in usage between high and low valued rows. Also, the term frequency is low to be effectively used for chi-squared test.